# Schoology Assignment Data

## Settings

In [1]:
from dotenv import load_dotenv
load_dotenv()

API_KEY = os.getenv("SCHOOLOGY_KEY")
API_SECRET = os.getenv("SCHOOLOGY_SECRET")

# Ed-Fi/MSDF users may have this env var set, which causes problems and is unnecessary for the code below
os.environ["REQUESTS_CA_BUNDLE"] = ""

## Establish connection to the API

In [2]:
import schoolopy
from prettyprinter import pprint

sc = schoolopy.Schoology(schoolopy.Auth(API_KEY, API_SECRET), verbose = True)

## Grading periods

Sections belong to grading periods. Past grading periods are not included in the API results, but future ones are. A section might be associated with multiple grading periods, e.g both a quarter or 6-week period and a semester, or even a year. Automatically determining which grading period(s) to use seems ripe for bugs. Therefore, it may be most useful to provide a utility for looking up grading period IDs, and then putting those IDs into the `.env` file before running an extract process.

In [3]:
print(sc.get_grading_periods())

--> calling https://api.schoology.com/v1/gradingperiods?limit=20&start=0
[GradingPeriod({'id': 825792, 'title': 'Fall Semester 2021-2022', 'start': '2021-09-01', 'end': '2021-12-20', 'active': 0, 'links': {'self': 'https://api.schoology.com/v1/gradingperiods/825792'}, 'has_children': False, 'parent': '0'}), GradingPeriod({'id': 825790, 'title': 'Spring Semester 2020-2021', 'start': '2021-01-11', 'end': '2021-06-04', 'active': 0, 'links': {'self': 'https://api.schoology.com/v1/gradingperiods/825790'}, 'has_children': False, 'parent': '0'}), GradingPeriod({'id': 822639, 'title': 'Fall Semester 2020-2021', 'start': '2020-08-28', 'end': '2020-12-18', 'active': 1, 'links': {'self': 'https://api.schoology.com/v1/gradingperiods/822639'}, 'has_children': False, 'parent': '0'}), GradingPeriod({'id': 825791, 'title': 'Spring Semester 2019-2020', 'start': '2020-01-13', 'end': '2020-06-05', 'active': 0, 'links': {'self': 'https://api.schoology.com/v1/gradingperiods/825791'}, 'has_children': False,

The `.env` file will have an item `SCHOOLOGY_GRADING_PERIODS` that contains a comma-separated list of grading periods of interest.

In [4]:
valid_grading_periods = os.getenv("SCHOOLOGY_GRADING_PERIODS")
assert valid_grading_periods is not None, "One or more grading periods needs to be defined in CSV format in the .env file, using key SCHOOLOGY_GRADING_PERIODS"

try:
    valid_grading_periods = [int(gp) for gp in valid_grading_periods.split(',')]
except ValueError as ex:
    msg = f"Grading period environment variable could not be parsed: {ex}"
    raise Exception(msg)

## Lists of courses and sections

The following returns 403 forbidden: `section = sc.get_sections()`. The URL used was `https://api.schoology.com/v1/sections`.

Instead, we need to query for sections from within courses, with URL like `https://api.schoology.com/v1/courses/%course_id/sections`. Will add a method into the API to support asking a course for its sections. Because of the way that schoolopy creates models, this new method is most easily added as an extension (aka monkey patch).

In [5]:
courses = sc.get_courses()

# Extend the Course object with a new method for retrieving course sections
from schoolopy.models import Section
def get_sections_for_course(Self):
    resource = "courses/{}/sections".format(Self.id)

    # TODO: make sure this handles situation where there are no sections for a course
    return [Section(raw) for raw in sc.get(resource)['section']]

schoolopy.models.Course.get_sections = get_sections_for_course
del get_sections_for_course
# End extension

sections = []
for c in courses:
    # Filter only sections for the grading periods we care about.
    sections += [c for c in c.get_sections() for gp in valid_grading_periods if [gp] < c.grading_periods]

print(sections)
print(f"There are {len(sections)} sections")

--> calling https://api.schoology.com/v1/courses?limit=20&start=0
--> calling https://api.schoology.com/v1/courses/2941242684/sections?limit=20&start=0
--> calling https://api.schoology.com/v1/courses/2942191514/sections?limit=20&start=0
[Section({'id': '2975852079', 'course_title': 'Algebra I', 'course_code': 'ALG-1', 'course_id': '2942191514', 'school_id': '2908525646', 'access_code': 'XD43P-TBX6938C', 'section_title': 'Section 2', 'section_code': '', 'section_school_code': '', 'synced': '0', 'active': 1, 'description': '', 'parent_id': None, 'grading_periods': [825792], 'profile_url': 'https://api.schoology.com/sites/all/themes/schoology_theme/images/course-default.svg', 'location': '', 'meeting_days': [''], 'start_time': '', 'end_time': '', 'weight': '0', 'options': {'weighted_grading_categories': '0', 'upload_documents': '0', 'create_discussion': '0', 'member_post': '1', 'member_post_comment': '1', 'default_grading_scale_id': 0, 'content_index_visibility': {'topics': 0, 'assignmen

## List of all students

The call `sc.get_users()` returns all users, not just students. A new `get_students` method was added to the schoolopy module.

In [6]:
students = sc.get_students()

print(f'Count: {len(students)}. First student:')
pprint(students[0])

--> calling https://api.schoology.com/v1/roles?limit=20&start=0
--> calling https://api.schoology.com/v1/users?limit=20&start=0
Count: 6. First student:
schoolopy.models.User({
    'uid': '100032890',
    'id': 100032890,
    'school_id': 2908525646,
    'synced': 0,
    'school_uid': '604863',
    'name_title': '',
    'name_title_show': 0,
    'name_first': 'Mary',
    'name_first_preferred': '',
    'use_preferred_first_name': '1',
    'name_middle': '',
    'name_middle_show': 0,
    'name_last': 'Archer',
    'name_display': 'Mary Archer',
    'username': 'mary.archer',
    'primary_email': 'mary.archer@studentgps.org',
    'picture_url':
        'https://asset-cdn.schoology.com/system/files/imagecache/profile_reg/'
        'sites/all/themes/schoology_theme/images/user-default.gif',
    'gender': None,
    'position': None,
    'grad_year': '',
    'password': '',
    'role_id': 796380,
    'tz_offset': -5,
    'tz_name': 'America/Chicago',
    'parents': None,
    'child_uids': N

## List of all assignments

Assignments are retrieved for sections. 

In [14]:
# Recall that we already have a `sections` object
print(len(sections))

assignments = []

for s in sections:
    assignments += sc.get_assignments(s.id)

print(assignments)

2
--> calling https://api.schoology.com/v1/sections/2941242697/assignments?limit=20&start=0
--> calling https://api.schoology.com/v1/sections/2942191527/assignments?limit=20&start=0
[Assignment({'id': 2942237212, 'title': 'Anna Karenina - Character Study', 'description': "Write an essay summarizing character attributes, including comparative analysis where appropriate, for Tolstoy's <i>Anna Karenina</i>.", 'due': '2020-08-21 23:59:00', 'grading_scale': '0', 'grading_period': '822639', 'grading_category': '38619916', 'max_points': '100', 'factor': '1', 'is_final': '0', 'show_comments': '0', 'grade_stats': '0', 'allow_dropbox': '1', 'allow_discussion': '1', 'published': 1, 'type': 'assignment', 'grade_item_id': 2942237212, 'available': 1, 'completed': 0, 'dropbox_locked': 0, 'grading_scale_type': 0, 'show_rubric': False, 'display_weight': '0', 'folder_id': '0', 'assignment_type': 'basic', 'web_url': 'https://app.schoology.com/assignment/2942237212', 'num_assignees': 0, 'assignees': [], '

## List of homework submissions

The resource URL for a submission depends on `grade_item_id`, but what is this? We can see it in the assignment results above, and it also exists on Discussions. This appears to allow a submission grade to be associated with an Assignment or a Discussion.

Use the extension-method approach as done with the `get_sections` method, this time modifying `Assignment`.

In [15]:
# Extend the Section object with a new method for retrieving course sections
from schoolopy.models import Assignment
from schoolopy.models import Submission

def _get_section_id(self):
    """
    The section ID is not directly stored in an Assignment, but it will be needed
    for getting submissions. Extract it from the `links.self` property.
    """

    # Could use a regex, but probably faster to split by / and just count over
    # to the correct index.
    return self.links['self'].split('/')[5]

Assignment._get_section_id = _get_section_id
del _get_section_id

def get_submissions(self):
    resource = "sections/{}/submissions/{}".format(self._get_section_id(), self.grade_item_id)

    # TODO: make sure this handles situation where there are no submissions
    return [Submission(raw) for raw in sc.get(resource)['revision']]

Assignment.get_submissions = get_submissions
del get_submissions
# End extension    

submissions = []
for a in assignments:
    submissions += a.get_submissions()

print(submissions)

--> calling https://api.schoology.com/v1/sections/2941242697/submissions/2942237212?limit=20&start=0
--> calling https://api.schoology.com/v1/sections/2941242697/submissions/2942243563?limit=20&start=0
--> calling https://api.schoology.com/v1/sections/2942191527/submissions/2942251001?limit=20&start=0
--> calling https://api.schoology.com/v1/sections/2942191527/submissions/2942255624?limit=20&start=0
[Submission({'revision_id': 1, 'uid': 100032890, 'created': 1598631545, 'num_items': 1, 'late': 1, 'draft': 0}), Submission({'revision_id': 1, 'uid': 100032891, 'created': 1598631744, 'num_items': 1, 'late': 1, 'draft': 0}), Submission({'revision_id': 1, 'uid': 100032895, 'created': 1598631616, 'num_items': 1, 'late': 1, 'draft': 0}), Submission({'revision_id': 1, 'uid': 100032896, 'created': 1598631955, 'num_items': 1, 'late': 1, 'draft': 0}), Submission({'revision_id': 1, 'uid': 100032897, 'created': 1598632008, 'num_items': 1, 'late': 1, 'draft': 0}), Submission({'revision_id': 1, 'uid'

In the results above, note the concept of "revision_id". A student can submit multiple versions. Also note that we have a draft status. We must decide what to do with that information - do we return all revisions? Or just the latest? Or perhaps only completed work? For now, going to return all available data.

Although there _are_ draft submissions, none are included in the result set. Reason unknown. However, we have decided that is acceptable - only need to report on the assignments that _have_ been submitted.

### TODO:
* Paging support